# 基板の設計
キーボードの基板を設計します。通常の工程で基板を作るにはまず`回路図エディタ`で`回路図`を書いて使用する部品とその結線情報をまとめて`ネットリスト`として出力し、`基板エディタ`を用いて実際に`基板`へ部品を配置・配線します。

# skidlでネットリスト(回路図)を設計する
Pythonを使った設計では回路図を書かずにコードで使用する部品と結線情報を定義してネットリストを出力します。[skidl](https://github.com/devbisme/skidl)というライブラリを用います。このライブラリは独自フォーマットの部品ライブラリだけでなく、kicadの部品ライブラリをインポートして使用します。また、ネットリストはkicadのPCBエディタと互換性があり直接kicadへ読み込んで回路図を書くこともできます。

- skidl  https://github.com/devbisme/skidl
- skidlのドキュメント  https://devbisme.github.io/skidl/

早速、`skidl`をインストールして設計をはじめましょう。

## skidlのインストールとインポート
ノートブック上に[skidl](https://github.com/devbisme/skidl)をインストールしてインポートしておきます。
このとき、インストールとインポートは別のセルにしておくと何度もインストールしなくて済むのでセルを分けておきます。
インストールには少し時間がかかるかもしれません。

In [ ]:
!pip install skidl

In [ ]:
from skidl import *
# kicadのデフォルトパスが環境変数に設定されていないのでWarningが出ます

## 定数を宣言しておく
何度も出てくる値を定数として宣言しておきます。この値を使い回すことで変更に強い設計になります。


In [ ]:
KEY_COUNT = 10
COL_COUNT = 4
ROW_COUNT = 3
MATRIX_MAP = [
        (0,1),(0,2),(0,3),  
        (1,1),(1,2),(1,3),
  (2,0),(2,1),(2,2),(2,3)
]

## kicadライブラリの追加
使用するkicadライブラリの追加をします。

kicadがインストールされている環境であれば通常のライブラリへのパスがデフォルトで追加されているはずですが、今回はkicadのインストールされていない環境で実行するために使う部品だけ別に用意しておきます。

回路図用のシンボル、基板用のフットプリント共に必要です。

次のセルを実行するとgithubリポジトリをクローンして必要なファイルをノートブックから読み出せるように準備します。


In [ ]:
!git clone https://github.com/hsgw/keyboard-made-by-python/

## kicadライブラリの読み込みと部品の準備
kicadライブラリを読み込んでシンボルとフットプリントを結びつけ、必要な個数だけ部品を用意します。

In [ ]:
# 使用するライブラリを登録する
# シンボル・フットプリントどちらも別で登録する
lib_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs")
footprint_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs/kicad.pretty")

# 同じ部品を何回も使うならシンボルとフットプリントを結びつけたテンプレートとして読み込んでおく
diode = Part(
  "kicad_symbols", "D_Small_ALT", TEMPLATE, footprint="kicad:D_SOD123_hand"
)
switch = Part(
  "kicad_symbols",
  "SW_Push",
  TEMPLATE,
  footprint="kicad:SW_Cherry_MX_1.00u_PCB",
)

# 宣言した定数とテンプレートを使ってダイオードとスイッチを用意する
# それぞれ KEY_COUNT 個用意して配列にいれておく
diodes = diode(KEY_COUNT)
switches = switch(KEY_COUNT)

# ひとつしか使わない部品はそのまま読み込む
xiao = Part("kicad_symbols", "xiao_rp2040", footprint="kicad:xiao_rp2040")
oled = Part("kicad_symbols", "oled_i2c", footprint="kicad:oled_i2c")

# printするとシンボルのピン情報が見えます
print(diode, switch, xiao, oled)

## ピンやネットを接続する
用意した部品のピン同士を接続を入力して回路の構成を定義します。

このときなるべく配線自体に`Net`名をつけるようにします。`Net`を基準にピンを繋いでいくことで同じところにつながるもの(電源やスイッチマトリクス)を整理して見やすいコードにします。また、kicadへネットリストをインポートしたときに表示されるので目印にもなります。

今回の設計でも基板配線時にピン番号を`Net`から読み出します。

In [ ]:
# ピン情報を表示しながら配線すると便利
print(xiao)

In [ ]:
# スイッチマトリクスのROW,COLのネットリストが入った配列を作る
netRows = [Net(f"ROW{i}") for i in range(ROW_COUNT)]
netCols = [Net(f"COL{i}") for i in range(COL_COUNT)]


# ROW -> スイッチの1ピン -> スイッチの2ピン -> ダイオードのカソード -> ダイオードのアノード -> COL をまとめて宣言する
# Netやピンは & で繋ぐと接続される
# 部品のピンには part[pin no]でアクセス出来る
# 部品のピンの添字を2つにするとinとoutになる
# sw["1"]につながるNetかpin & sw["1 2"] & sw["2"]に繋がるNetかpin
for sw, d, mapping in zip(switches, diodes, MATRIX_MAP):
  netRows[mapping[0]] & sw["1 2"] & d["K A"] & netCols[mapping[1]]

# スイッチマトリクスとxiaoを接続する
netCols[0] += xiao[8]
netCols[1] += xiao[3]
netCols[2] += xiao[4]
netCols[3] += xiao[5]

netRows[0] += xiao[1]
netRows[1] += xiao[6]
netRows[2] += xiao[7]

# oledとxiaoも接続する
# Net("3.3V")みたいに直接宣言して繋いでもOK
Net("3.3V") & oled["Vcc"] & xiao["3.3V"]
Net("GND") & oled["GND"] & xiao["GND"]
Net("SDA") & oled["SDA"] & xiao[9]
Net("SCL") & oled["SCL"] & xiao[11]

# printすると接続されているピンが表示される
print(netRows[0])

## ERCとネットリストの出力
ERC(Electrical Rule Check・回路図のルールチェック)を書けて、ネットリストを出力します。

In [ ]:
# 他のセルを複数回実行しているとエラーが出たり回路が複数個になったりするかもしれません
# そのときは`メニュー`の`ランタイム -> ランタイムを再起動`して最初からやりなおすか`再起動してすべてのセルを実行`してください
ERC()
generate_netlist(file_="keyboard.net")

出力されたネットリストはkicadのpcbnewのネットリスト読み込みからインポートしてそのまま基板を作ることも出来ます。